In [9]:
!pip install numpy
!pip install pandas
!pip install google-generativeai
!pip install openai

In [ ]:
import pandas as pd
df=pd.read_csv("C:/Users/Praisy/Downloads/archive/tripadvisor_hotel_reviews.csv")
print(df.columns)


Index(['Review', 'Rating'], dtype='object')


In [13]:
def cl(df):
    cldf=df[['Review','Rating']]
    cldf = cldf.dropna()
    cldf=cldf.drop(columns=['Rating'])
    return cldf
cldf=cl(df)
cldf.to_csv('cleanhotel.csv', index=False)
print(cldf.head())

                                              Review
0  nice hotel expensive parking got good deal sta...
1  ok nothing special charge diamond member hilto...
2  nice rooms not 4* experience hotel monaco seat...
3  unique, great stay, wonderful time hotel monac...
4  great stay great stay, went seahawk game aweso...


In [ ]:
!pip install python-dotenv
pip install tenacity


In [1]:
import pandas as pd
import numpy as np
def red(inp, out, s):
    df=pd.read_csv(inp)
    if s>=len(df):
        df.to_csv(out, index=False)
        return
    sdf = df.sample(n=s, random_state=42)
    sdf.to_csv(out, index=False)
if __name__=="__main__":
    inp='cleanhotel.csv'
    out='rhotel.csv'
    s=5
    red(inp, out, s)

In [2]:
import os
from openai import OpenAI
import pandas as pd
from typing import Tuple
from dotenv import load_dotenv
import time
from tenacity import retry, stop_after_attempt, wait_exponential
load_dotenv()
openai=OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key='yourapi'
)
@retry(
    wait=wait_exponential(multiplier=1,min=4,max=60),  
    stop=stop_after_attempt(3)  
)
def analyze_sentiment(text: str) -> Tuple[str, float]:
    try:
        time.sleep(10)  
        completion = openai.chat.completions.create(
            model="google/gemini-2.0-flash-exp:free",
            messages=[
                {
                    "role": "system",
                    "content": "You are a sentiment analysis expert. Analyze the sentiment of the given text and respond with only : sentiment_label (positive, negative, or neutral) "
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        pred_sent = completion.choices[0].message.content.strip()
        return pred_sent
    except Exception as e:
        print(f"Error analyzing sentiment for text: {e}")
        raise  

def process_in_batches(df: pd.DataFrame, batch_size: int = 10) -> pd.DataFrame:
    df['sentiment_label'] = ''  
    total_reviews = len(df)
    
    for start_idx in range(0, total_reviews, batch_size):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch = df.iloc[start_idx:end_idx]
        
        print(f"\nBatch {start_idx//batch_size + 1} ({start_idx}to{end_idx})")
        
        for idx, row in batch.iterrows():
            try:
                label = analyze_sentiment(row['Review'])  
                df.at[idx, 'pred_sent'] = label
                print(f"review {idx} : {label}")
                if (idx + 1) % batch_size == 0:
                    df.to_csv('sentanalysis.csv', index=False)
                    print(f"Saved till {idx + 1}")   
            except Exception as e:
                print(f"Failed to process {idx}: {e}")
                df.to_csv('sentanalysis.csv', index=False)
                print("Progress saved due to error")
                continue
        
        time.sleep(500)  
    
    return df

if __name__ == "__main__":
    try:
        if os.path.exists('sentanalysis.csv'):
            print("Found existing file. Resuming...")
            df_with_sentiment = pd.read_csv('sentanalysis.csv')
            remaining_reviews = df_with_sentiment[df_with_sentiment['pred_sent'] == '']
            if len(remaining_reviews) > 0:
                print(f"Continuing with {len(remaining_reviews)} remaining reviews...")
                df_with_sentiment = process_in_batches(remaining_reviews, batch_size=5)
            else:
                print("Completed!")
        else:
            print("Processing reviews")
            df_with_sentiment = pd.read_csv('reduced_hotel.csv')
            df_with_sentiment = process_in_batches(df_with_sentiment, batch_size=5)
        output_file = 'predicted.csv'
        df_with_sentiment.to_csv(output_file, index=False)
        print(f"\n{output_file} is the final File saved")
        print(df_with_sentiment.head())
    except Exception as e:
        print(f"An error occurred: {e}")


Processing reviews

Batch 1 (0to5)
review 0 : negative
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
review 1 : positive
review 2 : positive
review 3 : positive
review 4 : positive
Saved till 5

Batch 2 (5to10)
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
Failed to process 5: RetryError[<Future at 0x12b6731cf40 state=finished raised TypeError>]
Progress saved due to error
review 6 : positive
review 7 : negative
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
review 8 : negative
Error analyzing sentiment for text: 'NoneType' object is not subscriptable
review 9 : positive
Saved till 10

predicted.csv is the final File saved
                                              Review  Rating sentiment_label  \
0  not recommend hotel did reviewers actually sta... 

In [3]:
import os
from openai import OpenAI
import pandas as pd
from typing import Tuple
from dotenv import load_dotenv
import time
from tenacity import retry, stop_after_attempt, wait_exponential

# Load environment variables
load_dotenv()

# Initialize OpenRouter client
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key='yourapi'
)

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),  # Wait between retries, increasing exponentially
    stop=stop_after_attempt(5)  # Stop after 5 attempts
)
def analyze_sentiment(text: str) -> Tuple[str, float]:
    """
    Analyze the sentiment of a given text using OpenRouter API with Gemini model.
    Returns a tuple of (sentiment_label, confidence_score)
    """
    try:
        # Add delay between requests to respect rate limits
        time.sleep(10)
        
        completion = openai.chat.completions.create(
            model="google/gemini-2.0-flash-exp:free",
            messages=[
                {
                    "role": "system",
                    "content": "You are a sentiment analysis expert. Analyze the sentiment of the given text and respond with only two values: sentiment_label (positive, negative, or neutral) and confidence_score (between 0 and 1) separated by a comma."
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        response = completion.choices[0].message.content
        sentiment_label, confidence_score = response.strip().split(',')
        confidence_score = float(confidence_score)
        
        return sentiment_label, confidence_score
    except Exception as e:
        print(f"Error analyzing sentiment for text: {e}")
        raise

def manual_review_input():
    
    print("Enter a review for sentiment analysis (Type 'exit' to stop): ")
    while True:
        review = input("Review: ")
        
        if review.lower() == 'exit':
            print("Exiting the sentiment analysis.")
            break
        
        try:
            label, score = analyze_sentiment(review)
            print(f"Sentiment: {label}, Confidence Score: {score:.2f}")
            
        except Exception as e:
            print(f"An error occurred while analyzing the review: {e}")

if __name__ == "__main__":
    try:
        manual_review_input()
    except Exception as e:
        print(f"An error occurred: {e}")


Enter a review for sentiment analysis (Type 'exit' to stop): 


Sentiment: positive, Confidence Score: 0.70
Sentiment: negative, Confidence Score: 0.70
Exiting the sentiment analysis.


In [4]:
import os
from openai import OpenAI
import pandas as pd
from typing import Tuple
from dotenv import load_dotenv
import time
from tenacity import retry, stop_after_attempt, wait_exponential

load_dotenv()
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key='yourapi'
)

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),  
    stop=stop_after_attempt(5)  
)
def analyze_sentiment(text: str) -> Tuple[str, float]:
    try:
 
        time.sleep(10)  
        
        completion = openai.chat.completions.create(
            model="google/gemini-2.0-flash-exp:free",
            messages=[
                {
                    "role": "system",
                    "content": "You are a sentiment analysis expert. Analyze the sentiment of the given text and respond with only two values: sentiment_label (positive, negative, or neutral) and confidence_score (between 0 and 1) separated by a comma."
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        response = completion.choices[0].message.content
        sentiment_label, confidence_score = response.strip().split(',')
        confidence_score = float(confidence_score)
        
        return sentiment_label, confidence_score
    except Exception as e:
        print(f"Error analyzing sentiment for text: {e}")
        raise  
def single_review(review_text: str):
    label, score = analyze_sentiment(review_text)
    print(f"Sentiment: {label}, Score: {score:.2f}")
if __name__ == "__main__":
    try:
        review = input("Enter a review for sentiment analysis: ")
        single_review(review)
    except Exception as e:
        print(f"An error occurred: {e}")


Sentiment: positive, Score: 0.70
